# Описание задачи

https://drive.google.com/drive/folders/190bq-AUJTwMza179pVlH_FMDdA5sOjup?usp=share_link

https://ods.ai/competitions/nto22-23/

Второй этап НТО ИИ

https://ntcontest.ru/docs/ai-assignements1.pdf

# Загрузка данных и библиотек

In [ ]:
!pip install einops
!pip install timm
!pip install -q rich
!pip install -q wandb --upgrade;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import sys
import re
import gc
import platform
import random
import argparse

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import einops

import albumentations as A
from albumentations.pytorch import ToTensorV2

import timm
import glob
import cv2

from numpy import array
from numpy import argmax

from tqdm import tqdm
from tqdm.contrib import tzip

import wandb

import warnings

warnings.simplefilter('ignore')

In [ ]:
# это только если используете wandb
# https://wandb.ai/
# если нет, то не надо
def wandb_log(**kwargs):
    """
    Logs a key-value pair to W&B
    """
    for k, v in kwargs.items():
        wandb.log({k: v})

In [ ]:
# это только если используете wandb
# https://wandb.ai/
# если нет, то не надо
# Log in to your W&B account
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:
timm.list_models()

In [ ]:
# это только если используете wandb
# https://wandb.ai/
# если нет, то не надо
conf = dict(
    MAX_FRAMES=12,
    EPOCHS=10,#7
    LR=2e-4,
    IMG_SIZE=(224, 224),
    FEATURE_EXTRACTOR='resnext101_64x4d',
    DR_RATE=0.3,
    NUM_CLASSES=12,
    RNN_HIDDEN_SIZE=256,
    RNN_LAYERS=1,
    TRAIN_BS=7, # количество батчей на трейне
    VALID_BS = 7, # количество батчей на валидации
    NUM_WORKERS = 4, # количество процессоров которые используются

    _wandb_kernel = 'tanaym',
    MODEL_PATH='/content/gdrive/MyDrive/Classroom/model_resnext50_32x4d.pt',
    video_path='/content/gdrive/MyDrive/Classroom/train_video'
    )

run = wandb.init(
    project='pytorch',
    config=conf,
    job_type='train',
)

batch_train_loss,█▇▇▅▇▄▄▄▃▂▂▃▁▃▃▁▂▁▁▁▇▆▅▃▄▃▃▅▃▂▂▁▁▂▁▁▁▁▁▁
batch_val_loss,▇▆▄▆▇▄▆▇▅▆▄▄▅▄▄▂▃▃▂▃▂▂▃▁▃▁▅▄▁▂▅▂▄▁▄▅█▄▁▃
train_loss,███▆▅▄▃▂▂▁▁▁█▅▄▃▂▂▂▁▁▁
valid_loss,███▄▂▂▁▂▁▂▃▃
batch_train_loss,2.14652
batch_val_loss,2.27924
train_loss,0.10104
valid_loss,1.08348


In [ ]:
# Монтирует google disk в colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
url = os.chdir(r"/content/gdrive/MyDrive/Classroom/train_video")
os.getcwd()

'/content/gdrive/.shortcut-targets-by-id/1giDWS5DoXM--4LqJomIvI5TocoCWYyNe/train_video'

In [ ]:
os.listdir()

In [ ]:
df_train1 = pd.read_csv('/content/gdrive/MyDrive/Classroom/train.csv', index_col=0)
df_train2=  pd.read_csv('/content/gdrive/MyDrive/Classroom/test - Лист1.csv')

df_train=pd.concat([df_train1, df_train2])

df_train.head()

,path,labels
578,France Argentiere church and cross with Alps_p...,cloud
229,Georgia Okefenokee reflections in scummy water...,water
544,170728_Berlin_A_012_preview.mp4,sunset
369,170209D_039_Diving_1080p_preview.mp4,water
442,motionplaces flower-Oct2018_preview.mp4,flower


In [ ]:
df_train=df_train.dropna(axis=0, how='any')

In [ ]:
set(df_train.path) - set(os.listdir('/content/gdrive/MyDrive/Classroom/train_video'))

set()

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 737 entries, 578 to 366
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    737 non-null    object
 1   labels  737 non-null    object
dtypes: object(2)
memory usage: 17.3+ KB


In [ ]:
df_train.shape

(584, 4)

In [ ]:
# Размелим датасет на обучающую и валидационную выборку, т.е. train_test_split, только самописный

df_valid = df_train.sample(frac=0.1) #берем рандомно 20% от исходного датасета
df_train = df_train[~df_train.index.isin(df_valid.index)]# теперь удалим эти 20% из исходного, это и останется на трейн
# df_valid = df_train
# df_train=df_train

df_train.reset_index(drop=True, inplace=True)
df_valid.reset_index(drop=True, inplace=True)

print(f'Train shape: {df_train.shape}')
print(f'Valid shape: {df_valid.shape}')

Train shape: (526, 4)
Valid shape: (58, 4)


In [ ]:
def read_video(path, img_size, transform=None, frames_num=8):
    frames = []
    cap = cv2.VideoCapture(path)

    fps = int(cap.get(cv2.CAP_PROP_FPS)) # число кадров в секунду
    # print(fps)

    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) # количество кадров
    # print(length)

    # мы берем не все кадры из видео, потому что их может быть много
    # а только frames_num (ну тут 6 написано)
    #  как мы берем? первые 6? последние 6? в середине 6?
    # нет берем кадры через определенный шаг, он вычисляется как N:
    N = length // (frames_num)

    current_frame = 1

    # в цикле по очереди считываем кадрый из видео
    for i in range(length):
        ret, frame = cap.read(current_frame)

        if ret and i == current_frame and len(frames) < frames_num:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # все кадры в BGR (поэтому перводим в RGB)
            frame = cv2.resize(frame, img_size)

            frames.append(frame)
            current_frame += N

    cap.release()

    # frames - это список состоящий из 6 трехмерных матрицы
    # 6 кадров
    # каждый кадр - это трех мерная матрица (потому что RGB)
    return frames

**Про класс DataSet:**

https://habr.com/ru/company/piter/blog/354912/

In [ ]:
# Класс для создания датасета
# он создается наследую базовый класс pytorch - Dataset
class RSNADataset(Dataset):
    # def __init__ - конструктор класс
    def __init__(self, df,  conf, is_test=False):
        # conf - это переменная в которой просто разные настройки лежат
        # она ниже определена
        self.conf = conf
        # df - это train.csv (где лежит название видео и его класс)
        self.df = df
        # transform - преобразование матриц кадров в тензоры (так как pytorch работает только с тензорами)
        # тензор по сути тоже самое что вектор (ну почти:) )
        self.transform = A.Compose([
            A.Normalize(),
            ToTensorV2(p=1.0),
        ],p=1.)

        self.is_test = is_test
        self.samples = []
        self.cls = {'water':0,'car':1,'cloud':2,'food':3,'flower':4,'dance':5,'animal':6,
                    'sunset':7,'fire':8,'cloud, sunset':2,'water, flower':0,'cloud, water':2}

        for path, label in tzip(df.path, df.labels):
            frames = read_video(self.conf['video_path'] + f'/{path}', self.conf['IMG_SIZE'], self.conf['MAX_FRAMES'])
            self.samples +=[{'label': self.cls[str(label)],'frames': frames}]

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # __getitem__ - вызывается когда вызывается DataLoader
        sample = self.samples[idx]

        frames = sample['frames']

        # for i in range(len(frames)):
        #      frames[i] = frames[i] / 255.0

        # if self.transform is not None:
        #     for i in range(len(frames)):
        #         frames[i] = self.transform(image=np.array(frames[i]))['image']

        # torch.from_numpy - перевод в тензоры
        frames_tr = torch.from_numpy(np.stack(frames, axis=2)).permute(2, 3, 0, 1)

        if self.is_test:
            return frames_tr
        else:
            label = torch.tensor(sample['label']).long()
            return frames_tr, label

In [ ]:
{'water':0,'car':1,'cloud':2,'food':3,'flower':4,'dance':5,'animal':6,
                    'sunset':7,'fire':8,'cloud, sunset':2,'water, flower':0,'cloud, water':2}

df_train["labels"].value_counts()

water            117
cloud             89
car               76
flower            60
sunset            57
food              55
animal            52
dance             40
cloud, water      23
fire              14
cloud, sunset      1
Name: labels, dtype: int64

In [ ]:
%%time

train_data = RSNADataset(df_train, conf)

train_loader = DataLoader(
        train_data,
        batch_size=7,
        shuffle=False
    )

valid_data = RSNADataset(df_valid, conf)

valid_loader = DataLoader(
        valid_data,
        batch_size=7,
        shuffle=False
    )

  0%|          | 0/526 [00:00<?, ?it/s]

  0%|          | 0/58 [00:00<?, ?it/s]

CPU times: user 4min 34s, sys: 16 s, total: 4min 50s
Wall time: 3min 10s


In [ ]:
# prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
# for batch, (frames, targets) in prog_bar:
#    print((frames, targets))

In [ ]:
print(f'Sample len (это эквивалентно количество видео): {len(train_data.samples)}') # всего в датасете 522 видео
print(f'Что лежит в одном видео: {len(train_data.samples[0])}') # в каждом видео лежат два ключа
print(f'Какие ключи: {train_data.samples[0].keys()}')# в каждом видео лежат два ключа
print(f'Что лежит в ключе labels: {train_data.samples[0]["label"]}')# ключ label
print(f'размерность данных по ключу frames: {len(train_data.samples[0]["frames"])}')# лежит 6, так как кадров из видео 6
print(f'Размерность одного кадра, frame: {train_data.samples[0]["frames"][0].shape}') # что лежит в одном кадре, его размерность

Sample len (это эквивалентно количество видео): 584
Что лежит в одном видео: 2
Какие ключи: dict_keys(['label', 'frames'])
Что лежит в ключе labels: 2
размерность данных по ключу frames: 6
Размерность одного кадра, frame: (224, 224, 3)


In [ ]:
## Model Class

class ResNextModel(nn.Module):
    def __init__(self, conf):
        super(ResNextModel, self).__init__()
        self.conf = conf
        self.backbone = timm.create_model(self.conf['FEATURE_EXTRACTOR'], pretrained=True)

    def forward(self, x):
        return self.backbone(x)


class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


class RSNAModel(nn.Module):
    def __init__(self, conf, pretrained=True):
        super(RSNAModel, self).__init__()
        self.conf = conf
        self.backbone = ResNextModel(self.conf)
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.num_features = self.backbone.backbone.fc.in_features

        self.backbone.backbone.fc = Identity()
        self.dropout = nn.Dropout(self.conf['DR_RATE'])
        self.rnn = nn.LSTM(self.num_features, self.conf['RNN_HIDDEN_SIZE'], self.conf['RNN_LAYERS'], dropout=0.1,
                           batch_first=True, bidirectional=True)  # bidirectional=True
        self.fc1 = nn.Linear(self.conf['RNN_HIDDEN_SIZE'] * 2, self.conf['NUM_CLASSES']) #выходной слой

    def forward(self, x):
        b, f, c, h, w = x.shape
        # print(x.reshape(b * f, c, h, w).shape)
        x = self.backbone(x.reshape(b * f, c, h, w))
        # print(x.shape)
        x = x.reshape(b, f, self.num_features)
        # print(x.shape)
        x, _ = self.rnn(x)
        # print(x.shape)
        x = self.dropout(x[:, -1])  #
        # print(x.shape, self.num_features)
        x = self.fc1(x)
        return x

In [ ]:
def train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Trains model for one epoch
    """
    model.train()
    running_loss = 0
    prog_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    for batch, (frames, targets) in prog_bar:
        optimizer.zero_grad()

        frames = frames.to(device, torch.float)
        targets = targets.to(device)

        # # Re arrange the frames in the format our model wants to recieve
        preds = model(frames)
        loss = loss_fn(preds, targets)

        loss.backward()
        optimizer.step()

        loss_item = loss.item()
        running_loss += loss_item

        prog_bar.set_description(f"loss: {loss_item:.4f}")

        if log_wandb == True:
            wandb_log(
                batch_train_loss=loss_item
            )

        if verbose == True and batch % 20 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")

    avg_loss = running_loss / len(train_dataloader)

    return avg_loss

@torch.no_grad()
def valid_one_epoch(model, valid_dataloader, loss_fn, epoch, device, log_wandb=True, verbose=False):
    """
    Validates the model for one epoch
    """
    model.eval()
    running_loss = 0
    prog_bar = tqdm(enumerate(valid_dataloader), total=len(valid_dataloader))
    for batch, (frames, targets) in prog_bar:
        frames = frames.to(device, torch.float)
        targets = targets.to(device)

        # Re arrange the frames in the format our model wants to recieve
        preds = model(frames)
        loss = loss_fn(preds, targets)

        loss_item = loss.item()
        running_loss += loss_item

        prog_bar.set_description(f"val_loss: {loss_item:.4f}")

        if log_wandb == True:
            wandb_log(
                batch_val_loss=loss_item
            )

        if verbose == True and batch % 10 == 0:
            print(f"Batch: {batch}, Loss: {loss_item}")

    avg_val_loss = running_loss / len(valid_dataloader)

    return avg_val_loss

In [ ]:
# Функция main, которая собирает все вместе
# она принимает разные конфигурации для выполнения всех программ
# лоадер данных - df_loader
# df - это датафрейм (где путь, класс)
# output_path - путь куда сохранить ответы
# device - GPU/CPU - на чем делать вычисления
def main(conf, train_loader, valid_loader,  output_path, device):

    # создаем модель
    model = RSNAModel(conf)
    # передаем модели на чем делать вычисления GPU/CPU
    model = model.to(device)

    # Тут начинается обучение модели
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=conf['LR'])
    # SGD

    train_loss_fn = nn.CrossEntropyLoss()
    valid_loss_fn = nn.CrossEntropyLoss()

    # train_loss_fn = nn.BCEWithLogitsLoss()
    # valid_loss_fn = nn.BCEWithLogitsLoss()

    print(f"\nUsing Backbone: {conf['FEATURE_EXTRACTOR']}")

    current_loss = 1000
    for epoch in range(conf['EPOCHS']):
        print(f"\n{'--'*8} EPOCH: {epoch+1} {'--'*8}\n")

        train_loss = train_one_epoch(model, train_loader, optimizer, train_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)

        valid_loss = valid_one_epoch(model, valid_loader, valid_loss_fn, epoch=epoch, device=device, log_wandb=log_wandb)

        print(f"val_loss: {train_loss:.4f}")

        if log_wandb == True:
            wandb_log(
                    train_loss=train_loss,
                    # valid_loss=valid_loss
            )
        # if train_loss < current_loss:
        #     current_loss = train_loss
        #     torch.save(model.state_dict(), f"model_{conf['FEATURE_EXTRACTOR']}.pt")

        if valid_loss < current_loss:
            current_loss = valid_loss
            torch.save(model.state_dict(), f"model_{conf['FEATURE_EXTRACTOR']}.pt")

    # тут происходит предсказание ответа
    gt = []
    pred = [] # массив куда будем сохранять ответы
    model.eval()

    # prog_bar - это просто чтобы красиво выводился процесс вычисления
    # в виде бегущей крассивой штукой и показом процента выполнения
    gt = []
    pred = []
    model.eval()
    prog_bar = tqdm(enumerate(valid_loader), total=len(valid_loader))
    for batch, (frames, targets) in prog_bar:
        frames = frames.to('cuda').float()
        gt +=  list(targets.numpy())
        with torch.no_grad():

            pred += list(model(frames).argmax(dim=-1).to('cpu').flatten().numpy())

    # # сохраняем ответы в датафрейм с путями в новую колонку labels_pred
    df_valid['labels_pred'] = pred

    # сохраняем ответы в файл csv
    os.makedirs(output_path, exist_ok=True)
    df_valid['labels_pred'].to_csv(output_path + 'predictions.csv')

    # gt = []
    # pred = [] # массив куда будем сохранять ответы
    # model.eval()

    # # prog_bar - это просто чтобы красиво выводился процесс вычисления
    # # в виде бегущей крассивой штукой и показом процента выполнения
    # prog_bar = tqdm(enumerate(train_loader), total=len(train_loader))
    # for batch, (frames, targets) in prog_bar:
    #     frames = frames.to('cuda').float()
    #     gt +=  list(targets.numpy())

    #     with torch.no_grad():
    #         pred += list(model(frames).argmax(dim=-1).to('cpu').flatten().numpy())

    # # сохраняем ответы в датафрейм с путями в новую колонку labels_pred
    # df_train['labels_pred'] = pred

    # # сохраняем ответы в файл csv
    # os.makedirs(output_path, exist_ok=True)
    # df_train['labels_pred'].to_csv(output_path + 'predictions.csv')

In [ ]:
os.chdir('/content/gdrive/MyDrive/Classroom')

In [ ]:
os.getcwd()

'/content/gdrive/MyDrive/Classroom'

In [ ]:
timm.list_models()

In [ ]:
%%time

# ВСЕ ЗАПУСКАЕМ
conf = dict(
    MAX_FRAMES=12,
    EPOCHS=10,#7
    LR=2e-4,
    IMG_SIZE=(224, 224),
    FEATURE_EXTRACTOR='resnext101_32x8d',
    DR_RATE=0.35,
    NUM_CLASSES=12,
    RNN_HIDDEN_SIZE=256,
    RNN_LAYERS=1,
    TRAIN_BS=7, # количество батчей на трейне
    VALID_BS = 7, # количество батчей на валидации
    NUM_WORKERS = 4, # количество процессоров которые используются

    _wandb_kernel = 'tanaym',
    MODEL_PATH='/content/gdrive/MyDrive/Classroom/submit/weights/model_resnext101_64x4d.pt',
    video_path='/content/gdrive/MyDrive/Classroom/train_video'
    )

log_wandb = True

if torch.cuda.is_available():
    print("Using GPU: {}\n".format(torch.cuda.get_device_name()))
    device = torch.device('cuda')
else:
    print("\nGPU not found. Using CPU: {}\n".format(platform.processor()))
    device = torch.device('cpu')

main(conf, train_loader, valid_loader, 'output/', device)

Using GPU: Tesla T4


Using Backbone: resnext101_32x8d

---------------- EPOCH: 1 ----------------



val_loss: 1.7062: 100%|██████████| 9/9 [00:06<00:00,  1.48it/s]


val_loss: 1.9748

---------------- EPOCH: 2 ----------------



val_loss: 1.4693: 100%|██████████| 9/9 [00:06<00:00,  1.43it/s]


val_loss: 1.5160

---------------- EPOCH: 3 ----------------



...

val_loss: 0.5699

---------------- EPOCH: 10 ----------------



val_loss: 1.4153: 100%|██████████| 9/9 [00:06<00:00,  1.45it/s]


val_loss: 0.4914


100%|██████████| 9/9 [00:06<00:00,  1.47it/s]

CPU times: user 11min 1s, sys: 4.01 s, total: 11min 5s
Wall time: 11min 12s


In [ ]:
classes = {'water':0,'car':1,'cloud':2,'food':3,'flower':4,'dance':5,'animal':6,
           'sunset':7,'fire':8,'cloud, sunset':2,'water, flower':0,'cloud, water':2}

df_train['labels_num'] = df_train['labels'].map(classes)

df_train

,path,labels,labels_num,labels_pred
0,France Argentiere church and cross with Alps_p...,cloud,2,2
1,Georgia Okefenokee reflections in scummy water...,water,0,0
2,170728_Berlin_A_012_preview.mp4,sunset,7,7
3,motionplaces flower-Oct2018_preview.mp4,flower,4,4
4,_import_6065c421dd66b8.88923114_preview.mp4,food,3,3
...,...,...,...,...
521,Nature 30 Seconds Timelapse.mp4,flower,4,4
522,rose flower time lapse shot slow motion shoote...,flower,4,4
523,Butterfly.mp4,flower,4,4
524,Red Admiral butterfly pollinating Asters in mi...,flower,4,4


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(df_valid['labels_num'].values, df_valid['labels_pred'].values))
print(f1_score(df_valid['labels_num'].values, df_valid['labels_pred'].values, average='weighted'))

'''0.6826923076923077
0.6732944381200194'''

0.5
0.47889036251105216


'0.6826923076923077\n0.6732944381200194'

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

conf_matrix = confusion_matrix(df_valid['labels_num'].values, df_valid['labels_pred'].values)

sns.heatmap(conf_matrix, annot = True, cmap="YlGnBu", linecolor='white', linewidths=1)

# Код по частям (ЭКСПЕРИМЕНТЫ)

**Про класс DataSet:**

https://habr.com/ru/company/piter/blog/354912/

По сути в samples лежит словарь для каждого видео, в словаре два ключа:
1. label - класс к каждому кадру из видео
2. frames - кадры из этого видео

https://learn.microsoft.com/en-us/windows/ai/windows-ml/tutorials/pytorch-train-model

https://paperswithcode.com/model/resnext?variant=resnext-50-32x4d

# Все единым кодом